In [1]:
import pandas
import datetime
import numpy
from sklearn import linear_model
import scipy
from scipy.optimize import curve_fit
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
from matplotlib import rcParams

In [2]:
# Some parameters we'll use
probCutOff = 0.2
modelVelScale = [ 0., 2000. ]
velsNormArr = numpy.arange(0.,2100.,100.) / max( modelVelScale )
dstVal = -100.

In [3]:
# SAPS Prob Pred Func
def saps_prob_pred_func(MLAT, normMLT, dst):    
    (a_sx, b_sx, a_sy, b_sy, a_xo, b_xo, a_yo, b_yo, a_o, b_o, theta) = \
        ( 3.11, 3.71e-03, 1.72,\
             8.19e-04, 4.59, 6.33e-02, -1.19, 3.21e-02,
             8.93e-01, -1.47e-03, 6.92e-01)
    x = MLAT - 57.5 # NORMALIZE
    y = normMLT
    sigma_x = a_sx + b_sx * dst
    sigma_y = a_sy + b_sy * dst
    xo = a_xo + b_xo * dst
    yo = a_yo + b_yo * dst
    amplitude = a_o + b_o * dst    
    
    a = (numpy.cos(theta)**2)/(2*sigma_x**2) + (numpy.sin(theta)**2)/(2*sigma_y**2)
    b = -(numpy.sin(2*theta))/(4*sigma_x**2) + (numpy.sin(2*theta))/(4*sigma_y**2)
    c = (numpy.sin(theta)**2)/(2*sigma_x**2) + (numpy.cos(theta)**2)/(2*sigma_y**2)
    prob = amplitude*numpy.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return prob

In [4]:
# SAPS Vel Azim Pred Func
def saps_velAzim_pred_func(normMLT, dst):

    ( a1mlt_azim, a2mlt_azim,\
               b1mlt_azim, b2mlt_azim ) = ( -1.18e+01, -4.55e-02, 4.24e-01, -6.13e-03 )
    
    # model shape parameters
    # mlt
    a_azim = a1mlt_azim + a2mlt_azim * dst
    b_azim = b1mlt_azim + b2mlt_azim * dst
    # func
    azim = a_azim + b_azim*(normMLT)
    
    return azim

In [5]:
# SAPS Vel Magn Pred Funcs
# We need to get the scale, shape and loc
# parameters and then use them get the pdf 
# and then get the mean velocity from the pdf!!!
def saps_vm_scale_pred_func(mlt,mlat, dst):
    
    ( a_ascmlt, b_ascmlt, a_bscmlt, \
         b_bscmlt, a_cscmlt, b_cscmlt,\
         a_asclat, b_asclat, a_bsclat, b_bsclat ) = ( -1.23e-02, -6.02e-05, -2.9e-04,\
                            -1.79e-05, -1.26e-04, -3.32e-06,\
                            3.47e+01, 2.2e-01, -8.09e-01, -9.76e-04 )
    
    
    a_scale_mlt = a_ascmlt + b_ascmlt * dst
    b_scale_mlt = a_bscmlt + b_bscmlt * dst
    c_scale_mlt = a_cscmlt + b_cscmlt * dst
    # mlat
    a_scale_mlat = a_asclat + b_asclat * dst
    b_scale_mlat = a_bsclat + b_bsclat * dst
    # func
    scale = ( a_scale_mlt + b_scale_mlt*(mlt) + c_scale_mlt*(mlt**2) ) * ( a_scale_mlat + b_scale_mlat*(mlat) )
    
    return scale


def saps_vm_shape_pred_func(mlt,mlat, dst):
    
    ( a_ashmlt, b_ashmlt, a_bshmlt, b_bshmlt,\
               a_ashlat, b_ashlat, a_bshlat, b_bshlat ) = ( -4.72e-02, -3.63e-04, -1.59e-03,\
                                     -2.3e-06, -5.61e+03, 1.19e+02, 8.42e+01, -2.05e+00 ) 
    
    # model shape parameters
    # mlt
    a_shape_mlt = a_ashmlt + b_ashmlt * dst
    b_shape_mlt = a_bshmlt + b_bshmlt * dst
    # mlat
    a_shape_mlat = a_ashlat + b_ashlat * dst
    b_shape_mlat = a_bshlat + b_bshlat * dst
    # func
    shape = ( a_shape_mlt + b_shape_mlt*(mlt) ) * ( a_shape_mlat + b_shape_mlat*(mlat) )
    
    return shape

def saps_vm_loc_pred_func(mlt,mlat, dst):
    
    
    ( a_alcmlt, b_alcmlt, a_blcmlt, b_blcmlt,\
               a_alclat, b_alclat, a_blclat, b_blclat ) = ( -2.81e-03, -2.11e-05, -1.01e-04, -5.51e-06,\
                                        -5.39e-01, 1.50e-03, 7.38e-02, -1.33e-04 )
    
    
    # model loc parameters
    # mlt
    a_loc_mlt = a_alcmlt + b_alcmlt * dst
    b_loc_mlt = a_blcmlt + b_blcmlt * dst
    # malt
    a_loc_mlat = a_alclat + b_alclat * dst
    b_loc_mlat = a_blclat + b_blclat * dst
    # func
    loc = ( a_loc_mlt + b_loc_mlt*(mlt) ) * ( a_loc_mlat*numpy.exp(b_loc_mlat*mlat) )
    return loc

def saps_velMagn_mean_pred_func( scale, shape, loc ):
    
    meanNormVel = scipy.stats.skewnorm.mean( shape, loc=loc, scale=scale )
    meanVel = meanNormVel * max( modelVelScale )
    return meanVel

In [6]:
colList = ["normMLT", "normLAT", "prob_pred",\
           "MLT", "MLAT", "dst_index", "shape_pred",\
           "scale_pred", "loc_pred", "mean_vel_pred", "azim_pred"]
sapsPredDF = pandas.DataFrame(columns=colList)
latArr = []
mltArr = []
normLatArr = []
normMltArr = []
dstValArr = []
dstBinArr = []


for x in numpy.arange(50., 65., 0.5):
    for y in range(-11,5):
        currNormLat =  x - 57.5
        normLatArr.append( currNormLat )
        normMltArr.append( y )
        if y > 0:
            mltArr.append( y )
        else:
            mltArr.append( y + 24 )
        latArr.append( x )
        dstValArr.append(dstVal)
        
sapsPredDF["MLT"] = mltArr
sapsPredDF["MLAT"] = latArr
sapsPredDF["normMLT"] = normMltArr
sapsPredDF["normLAT"] = normLatArr
sapsPredDF["dst_index"] = dstValArr



# Now calculate the prob of occ!!
sapsPredDF["prob_pred"] = saps_prob_pred_func( sapsPredDF["MLAT"],\
                                sapsPredDF["normMLT"], sapsPredDF["dst_index"] )
# Only keep those values whose Dst is gt 0.2
sapsPredDF = sapsPredDF[ sapsPredDF["prob_pred"] > 0.2 ].reset_index(drop=True)

# Now calculate the vel azim
sapsPredDF["azim_pred"] = saps_velAzim_pred_func( sapsPredDF["normMLT"], sapsPredDF["dst_index"] )


# Calculate vel mean magn now!
# We'll calculate scale, loc and shape
# Scale
sapsPredDF["scale_pred"] = saps_vm_scale_pred_func( sapsPredDF["normMLT"],\
                                        sapsPredDF["MLAT"], sapsPredDF["dst_index"] )
# loc
sapsPredDF["shape_pred"] = saps_vm_shape_pred_func( sapsPredDF["normMLT"],\
                                        sapsPredDF["MLAT"], sapsPredDF["dst_index"] )
# Shape
sapsPredDF["loc_pred"] = saps_vm_loc_pred_func( sapsPredDF["normMLT"],\
                                        sapsPredDF["MLAT"], sapsPredDF["dst_index"] )
# Finally get to the velocities
sapsPredDF["mean_vel_pred"] = saps_velMagn_mean_pred_func( sapsPredDF["scale_pred"],\
                                        sapsPredDF["shape_pred"], sapsPredDF["loc_pred"] )
# Round off all values to 2 decimal places!
sapsPredDF["prob_pred"] = sapsPredDF["prob_pred"].round(decimals=2)
sapsPredDF["mean_vel_pred"] = sapsPredDF["mean_vel_pred"].round(decimals=2)
sapsPredDF["azim_pred"] = sapsPredDF["azim_pred"].round(decimals=2)
# sapsPredDF.round
# Consider only those velocities whose magn are greater than 200 m/s
# sapsPredDF = sapsPredDF[ sapsPredDF["mean_vel_pred"] > 200.].reset_index(drop=True)
sapsPredDF.head()

,normMLT,normLAT,prob_pred,MLT,MLAT,dst_index,shape_pred,scale_pred,loc_pred,mean_vel_pred,azim_pred
0,-4,-5.5,0.24,20,52.0,-100,13.494936,0.218247,0.159649,666.62,-11.40
1,-3,-5.5,0.29,21,52.0,-100,16.856312,0.216838,0.130912,607.24,-10.36
2,-2,-5.5,0.27,22,52.0,-100,20.217688,0.205420,0.102175,531.75,-9.32
3,-5,-5.0,0.23,19,52.5,-100,9.540700,0.212717,0.196771,731.14,-12.44
4,-4,-5.0,0.35,20,52.5,-100,12.705420,0.221442,0.166755,685.79,-11.40


In [7]:
# Write the DF to a csv for plotting purposes
outCols = ["MLT", "MLAT", "prob_pred", "dst_index", "mean_vel_pred", "azim_pred"]
outFileName = "../data/full-model-sample-dst" + str(int(dstVal)) + ".txt"
sapsPredDF.to_csv(outFileName, columns = outCols, sep=' ', index=False, header=None)
print "saved to-->", outFileName

saved to--> ../data/full-model-sample-dst-100.txt


In [8]:
print sapsPredDF[ (sapsPredDF["mean_vel_pred"] > 700.) & \
                (sapsPredDF["normMLT"] > -4)]

    normMLT  normLAT  prob_pred  MLT  MLAT  dst_index  shape_pred  scale_pred  \
36       -3     -2.5       0.84   21  55.0       -100   10.939280    0.235887   
43       -3     -2.0       0.82   21  55.5       -100    9.953108    0.239062   
50       -3     -1.5       0.76   21  56.0       -100    8.966936    0.242237   
57       -3     -1.0       0.66   21  56.5       -100    7.980764    0.245412   
64       -3     -0.5       0.54   21  57.0       -100    6.994592    0.248587   
71       -3      0.0       0.42   21  57.5       -100    6.008420    0.251762   
72       -2      0.0       0.20   22  57.5       -100    7.206580    0.238506   
78       -3      0.5       0.31   21  58.0       -100    5.022248    0.254937   
84       -3      1.0       0.21   21  58.5       -100    4.036076    0.258112   

    loc_pred  mean_vel_pred  azim_pred  
36  0.170005         714.87     -10.36  
43  0.177572         734.72     -10.36  
50  0.185476         755.13     -10.36  
57  0.193732         776.

In [9]:
print sapsPredDF[ (sapsPredDF["normMLT"] == -8) ]

    normMLT  normLAT  prob_pred  MLT  MLAT  dst_index  shape_pred  scale_pred  \
52       -8     -1.0       0.22   16  56.5       -100    0.023404    0.140110   
59       -8     -0.5       0.25   16  57.0       -100    0.020512    0.141923   
66       -8      0.0       0.26   16  57.5       -100    0.017620    0.143735   
73       -8      0.5       0.25   16  58.0       -100    0.014728    0.145548   
79       -8      1.0       0.23   16  58.5       -100    0.011836    0.147361   
85       -8      1.5       0.21   16  59.0       -100    0.008944    0.149173   

    loc_pred  mean_vel_pred  azim_pred  
52  0.406365         817.96     -15.55  
59  0.424453         853.55     -15.55  
66  0.443346         890.73     -15.55  
73  0.463081         929.58     -15.55  
79  0.483693         970.17     -15.55  
85  0.505224        1012.58     -15.55  


In [10]:
print sapsPredDF[ (sapsPredDF["normMLT"] == -4) ]

    normMLT  normLAT  prob_pred  MLT  MLAT  dst_index  shape_pred  scale_pred  \
0        -4     -5.5       0.24   20  52.0       -100   13.494936    0.218247   
4        -4     -5.0       0.35   20  52.5       -100   12.705420    0.221442   
9        -4     -4.5       0.48   20  53.0       -100   11.915904    0.224638   
15       -4     -4.0       0.63   20  53.5       -100   11.126388    0.227833   
21       -4     -3.5       0.77   20  54.0       -100   10.336872    0.231029   
28       -4     -3.0       0.90   20  54.5       -100    9.547356    0.234225   
35       -4     -2.5       0.99   20  55.0       -100    8.757840    0.237420   
42       -4     -2.0       1.02   20  55.5       -100    7.968324    0.240616   
49       -4     -1.5       1.00   20  56.0       -100    7.178808    0.243811   
56       -4     -1.0       0.92   20  56.5       -100    6.389292    0.247007   
63       -4     -0.5       0.80   20  57.0       -100    5.599776    0.250203   
70       -4      0.0       0